# [Lab4] 모델 배포 및 테스트

이 노트북에서는 평가된 최고 성능 모델을 SageMaker 엔드포인트로 배포하고 테스트합니다.

## 주요 개선사항
- 안정적인 변수 관리
- 엔드포인트 상태 확인
- 견고한 테스트 로직
- 명확한 오류 처리

## 1. 환경 설정 및 변수 로드

In [1]:
# 이전 노트북에서 저장한 변수들 로드
%store -r

print("✅ 저장된 변수들을 로드했습니다.")

# 변수 존재 여부 확인
required_vars = ['model1_uri', 'model2_uri']
optional_vars = ['best_model_name', 'test_path']

missing_vars = []
for var in required_vars:
    if var not in globals():
        missing_vars.append(var)
    else:
        print(f"   ✅ {var}: {globals()[var]}")

for var in optional_vars:
    if var in globals():
        print(f"   ✅ {var}: {globals()[var]}")
    else:
        print(f"   ⚠️ {var}: 없음 (선택사항)")

if missing_vars:
    print(f"\n❌ 필수 변수 누락: {missing_vars}")
    print("   2-training.ipynb와 3-model-evaluation.ipynb를 먼저 실행해주세요.")
    READY_TO_DEPLOY = False
else:
    print("\n🚀 배포 준비 완료!")
    READY_TO_DEPLOY = True

✅ 저장된 변수들을 로드했습니다.
   ✅ model1_uri: s3://csv-file-store-6b806890/dzd-b6e9451edya6z4/3jxvlofqlqhfv4/dev/sagemaker/DEMO-xgboost-dm/output/conservative-xgb-training-2026-01-24-06-34-03-379/output/model.tar.gz
   ✅ model2_uri: s3://csv-file-store-6b806890/dzd-b6e9451edya6z4/3jxvlofqlqhfv4/dev/sagemaker/DEMO-xgboost-dm/output/aggressive-xgb-training-2026-01-24-06-34-06-147/output/model.tar.gz
   ✅ best_model_name: 적극적 하이퍼파라미터 모델
   ✅ test_path: s3://csv-file-store-6b806890/dzd-b6e9451edya6z4/3jxvlofqlqhfv4/dev/sagemaker/DEMO-xgboost-dm/test

🚀 배포 준비 완료!


In [2]:
# 필수 라이브러리 임포트
import sagemaker
import boto3
import pandas as pd
import numpy as np
import json
import time
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker_studio import Project

# AWS 세션 초기화
boto_session = boto3.Session()
sess = sagemaker.Session()
sm_client = boto3.client('sagemaker')
sm_runtime = boto3.client('sagemaker-runtime')

print("✅ 라이브러리 임포트 및 세션 초기화 완료")

sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
✅ 라이브러리 임포트 및 세션 초기화 완료


## 2. 프로젝트 설정

In [3]:
# 프로젝트 설정
project = Project()
role = project.iam_role
domain_id = project.domain_id
project_id= project.id

# 컨테이너 이미지 URI 가져오기
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.7-1'
)

print(f"✅ 프로젝트 설정 완료")
print(f"   - IAM 역할: {role}")
print(f"   - XGBoost 컨테이너: {container}")
print(f"   - 리전: {boto3.Session().region_name}")

✅ 프로젝트 설정 완료
   - IAM 역할: arn:aws:iam::891377361512:role/datazone_usr_role_3jxvlofqlqhfv4_cv8jraydalukr4
   - XGBoost 컨테이너: 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.7-1
   - 리전: us-west-2


## 3. 배포할 모델 선택

최고 성능 모델을 선택하거나 기본 모델을 사용합니다.

In [4]:
if READY_TO_DEPLOY:
    print("🏆 배포할 모델 선택")
    print("=" * 40)
    
    # 최고 성능 모델 확인
    if 'best_model_name' in globals() and best_model_name:
        print(f"✅ 최고 성능 모델 발견: {best_model_name}")
        
        if "보수적" in best_model_name or "conservative" in best_model_name.lower():
            selected_model_uri = model1_uri
            model_type = "conservative"
            print(f"   📊 모델 1 (보수적) 선택")
        else:
            selected_model_uri = model2_uri
            model_type = "aggressive"
            print(f"   📊 모델 2 (적극적) 선택")
    else:
        print("⚠️ 최고 성능 모델 정보 없음. 기본적으로 모델 1 사용")
        selected_model_uri = model1_uri
        model_type = "conservative"
        best_model_name = "보수적 하이퍼파라미터 모델 (기본값)"
    
    print(f"\n🎯 선택된 모델:")
    print(f"   - 이름: {best_model_name}")
    print(f"   - 타입: {model_type}")
    print(f"   - URI: {selected_model_uri}")
    
    # 전역 변수로 저장
    globals()['selected_model_uri'] = selected_model_uri
    globals()['model_type'] = model_type
    
else:
    print("❌ 필수 변수가 없어 모델 선택을 건너뜁니다.")

🏆 배포할 모델 선택
✅ 최고 성능 모델 발견: 적극적 하이퍼파라미터 모델
   📊 모델 2 (적극적) 선택

🎯 선택된 모델:
   - 이름: 적극적 하이퍼파라미터 모델
   - 타입: aggressive
   - URI: s3://csv-file-store-6b806890/dzd-b6e9451edya6z4/3jxvlofqlqhfv4/dev/sagemaker/DEMO-xgboost-dm/output/aggressive-xgb-training-2026-01-24-06-34-06-147/output/model.tar.gz


## 4. 모델 배포

선택된 모델을 SageMaker 엔드포인트로 배포합니다.

In [5]:
if READY_TO_DEPLOY and 'selected_model_uri' in globals():
    print("🚀 모델 배포 시작")
    print("=" * 40)
    
    # 타임스탬프 생성
    timestamp = strftime('%Y%m%d-%H%M%S', gmtime())
    
    # 모델 및 엔드포인트 이름 생성
    model_name = f"bank-marketing-{model_type}-{timestamp}"
    endpoint_name = f"bank-marketing-endpoint-{timestamp}"
    
    try:
        print(f"🔧 SageMaker 모델 생성 중: {model_name}")
        
        # SageMaker 모델 생성
        sagemaker_model = Model(
            model_data=selected_model_uri,
            image_uri=container,
            role=role,
            sagemaker_session=sess,
            name=model_name
        )
        
        print(f"✅ SageMaker 모델 생성 완료")
        
        print(f"\n🚀 엔드포인트 배포 중: {endpoint_name}")
        print(f"   ⏳ 배포 완료까지 5-10분 소요됩니다...")
        
        # 엔드포인트 배포 (비동기)
        predictor = sagemaker_model.deploy(
            initial_instance_count=1,
            instance_type='ml.m5.large',
            endpoint_name=endpoint_name,
            wait=False  # 비동기 배포
        )
        
        print(f"✅ 엔드포인트 배포 시작됨")
        print(f"   - 엔드포인트 이름: {endpoint_name}")
        print(f"   - 인스턴스: ml.m5.large × 1개")
        
        # 중요한 변수들을 전역으로 저장 (pickle 불가능한 객체는 제외)
        globals()['endpoint_name'] = endpoint_name
        globals()['model_name'] = model_name
        globals()['deployment_timestamp'] = timestamp
        
        DEPLOYMENT_STARTED = True
        
    except Exception as e:
        print(f"❌ 배포 실패: {e}")
        DEPLOYMENT_STARTED = False
        
else:
    print("❌ 배포 조건이 충족되지 않았습니다.")
    DEPLOYMENT_STARTED = False

# 배포 상태 저장
globals()['DEPLOYMENT_STARTED'] = DEPLOYMENT_STARTED

🚀 모델 배포 시작
🔧 SageMaker 모델 생성 중: bank-marketing-aggressive-20260124-081832
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
✅ SageMaker 모델 생성 완료

🚀 엔드포인트 배포 중: bank-marketing-endpoint-20260124-081832
   ⏳ 배포 완료까지 5-10분 소요됩니다...
✅ 엔드포인트 배포 시작됨
   - 엔드포인트 이름: bank-marketing-endpoint-20260124-081832
   - 인스턴스: ml.m5.large × 1개


## 5. 배포 상태 확인

엔드포인트 배포 상태를 확인하고 완료를 기다립니다.

In [6]:
if DEPLOYMENT_STARTED and 'endpoint_name' in globals():
    print(f"📊 엔드포인트 상태 확인: {endpoint_name}")
    
    try:
        # 엔드포인트 상태 조회
        response = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = response['EndpointStatus']
        
        print(f"\n🔍 현재 상태: {status}")
        print(f"   - 생성 시간: {response['CreationTime']}")
        
        if status == 'InService':
            print(f"✅ 엔드포인트 배포 완료!")
            ENDPOINT_READY = True
        elif status in ['Creating', 'Updating']:
            print(f"⏳ 배포 진행 중... 잠시 후 다시 확인해주세요.")
            print(f"   💡 다음 셀을 주기적으로 실행하여 상태를 확인하세요.")
            ENDPOINT_READY = False
        elif status == 'Failed':
            print(f"❌ 배포 실패!")
            if 'FailureReason' in response:
                print(f"   실패 이유: {response['FailureReason']}")
            ENDPOINT_READY = False
        else:
            print(f"⚠️ 알 수 없는 상태: {status}")
            ENDPOINT_READY = False
            
    except Exception as e:
        print(f"❌ 상태 확인 실패: {e}")
        ENDPOINT_READY = False
        
else:
    print("❌ 배포가 시작되지 않았습니다.")
    ENDPOINT_READY = False

# 상태 저장
globals()['ENDPOINT_READY'] = ENDPOINT_READY

📊 엔드포인트 상태 확인: bank-marketing-endpoint-20260124-081832

🔍 현재 상태: Creating
   - 생성 시간: 2026-01-24 08:18:36.574000+00:00
⏳ 배포 진행 중... 잠시 후 다시 확인해주세요.
   💡 다음 셀을 주기적으로 실행하여 상태를 확인하세요.


## 6. 테스트 데이터 준비

엔드포인트 테스트를 위한 데이터를 준비합니다.

In [7]:
print("📊 테스트 데이터 준비")

# 임시 디렉토리 생성
!mkdir -p tmp

import os

# 방법 1: 기존 테스트 데이터 사용 시도
test_data_loaded = False

if 'test_path' in globals() and test_path:
    try:
        print(f"📥 S3에서 테스트 데이터 다운로드 시도: {test_path}")
        !aws s3 cp $test_path/test_x.csv tmp/test_x.csv
        
        if os.path.exists('tmp/test_x.csv'):
            test_sample = pd.read_csv('tmp/test_x.csv', header=None, nrows=1)
            test_data = test_sample.values[0]
            test_data_loaded = True
            print(f"✅ 실제 테스트 데이터 로드 성공")
        
    except Exception as e:
        print(f"⚠️ S3 테스트 데이터 로드 실패: {e}")

# 방법 2: 로컬 파일 확인
if not test_data_loaded and os.path.exists('tmp/test_x.csv'):
    try:
        test_sample = pd.read_csv('tmp/test_x.csv', header=None, nrows=1)
        test_data = test_sample.values[0]
        test_data_loaded = True
        print(f"✅ 로컬 테스트 데이터 사용")
    except Exception as e:
        print(f"⚠️ 로컬 테스트 데이터 로드 실패: {e}")

# 방법 3: 샘플 데이터 생성
if not test_data_loaded:
    print("🔧 샘플 테스트 데이터 생성")
    
    # 은행 마케팅 데이터와 유사한 샘플 생성
    np.random.seed(42)  # 재현 가능한 결과
    
    # 59개 특성의 샘플 데이터 (원-핫 인코딩된 형태)
    sample_data = np.random.rand(59)
    
    # 일부 특성을 0/1로 설정 (원-핫 인코딩 특성들)
    sample_data[30:] = np.random.choice([0, 1], size=29)  # 범주형 특성들
    
    test_data = sample_data
    test_data_loaded = True
    
    # 파일로 저장
    pd.DataFrame([test_data]).to_csv('tmp/test_x.csv', header=False, index=False)
    print(f"✅ 샘플 데이터 생성 및 저장 완료")

if test_data_loaded:
    print(f"\n📋 테스트 데이터 정보:")
    print(f"   - 형태: {test_data.shape}")
    print(f"   - 첫 5개 값: {test_data[:5]}")
    print(f"   - 데이터 타입: {test_data.dtype}")
    
    # 전역 변수로 저장
    globals()['test_data'] = test_data
    globals()['test_data_loaded'] = True
    
else:
    print("❌ 테스트 데이터 준비 실패")
    globals()['test_data_loaded'] = False

📊 테스트 데이터 준비
📥 S3에서 테스트 데이터 다운로드 시도: s3://csv-file-store-6b806890/dzd-b6e9451edya6z4/3jxvlofqlqhfv4/dev/sagemaker/DEMO-xgboost-dm/test
download: s3://csv-file-store-6b806890/dzd-b6e9451edya6z4/3jxvlofqlqhfv4/dev/sagemaker/DEMO-xgboost-dm/test/test_x.csv to tmp/test_x.csv
✅ 실제 테스트 데이터 로드 성공

📋 테스트 데이터 정보:
   - 형태: (59,)
   - 첫 5개 값: [ 53   3 999   0   1]
   - 데이터 타입: int64


## 7. 엔드포인트 테스트

배포된 엔드포인트를 테스트합니다.

In [8]:
# 엔드포인트 테스트 실행
if (globals().get('ENDPOINT_READY', False) and 
    globals().get('test_data_loaded', False) and 
    'endpoint_name' in globals()):
    
    print(f"🧪 엔드포인트 테스트 시작: {endpoint_name}")
    print("=" * 50)
    
    try:
        # 테스트 데이터를 CSV 형태로 변환
        csv_data = ','.join(map(str, test_data))
        
        print(f"📊 예측 수행 중...")
        print(f"   - 입력 데이터 크기: {len(test_data)} 특성")
        print(f"   - 첫 3개 값: {test_data[:3]}")
        
        # SageMaker Runtime을 사용한 예측
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=csv_data
        )
        
        # 결과 파싱
        result = response['Body'].read().decode('utf-8')
        
        try:
            prediction = float(result.strip())
        except ValueError:
            # JSON 형태일 수 있음
            try:
                prediction = json.loads(result)
                if isinstance(prediction, list):
                    prediction = prediction[0]
            except:
                prediction = result
        
        print(f"\n✅ 예측 완료!")
        print(f"   - 원시 결과: {result}")
        print(f"   - 파싱된 예측값: {prediction}")
        
        if isinstance(prediction, (int, float)):
            probability = float(prediction)
            print(f"   - 확률: {probability:.4f}")
            print(f"   - 예측: {'정기예금 가입 가능성 높음' if probability > 0.5 else '정기예금 가입 가능성 낮음'}")
            
            # 추가 테스트 (5회)
            print(f"\n🔄 추가 테스트 (5회):")
            predictions = []
            
            for i in range(5):
                try:
                    response = sm_runtime.invoke_endpoint(
                        EndpointName=endpoint_name,
                        ContentType='text/csv',
                        Body=csv_data
                    )
                    result = response['Body'].read().decode('utf-8')
                    pred = float(result.strip())
                    predictions.append(pred)
                    print(f"   테스트 {i+1}: {pred:.4f}")
                except Exception as e:
                    print(f"   테스트 {i+1}: 실패 ({e})")
            
            if predictions:
                avg_pred = np.mean(predictions)
                std_pred = np.std(predictions)
                print(f"\n📊 테스트 결과 요약:")
                print(f"   - 평균 예측값: {avg_pred:.4f}")
                print(f"   - 표준편차: {std_pred:.4f}")
                print(f"   - 일관성: {'높음' if std_pred < 0.001 else '보통' if std_pred < 0.01 else '낮음'}")
        
        TEST_SUCCESS = True
        
    except Exception as e:
        print(f"❌ 예측 실패: {e}")
        print(f"\n🔍 문제 해결 방법:")
        print(f"   1. 엔드포인트 상태 재확인 (위의 상태 확인 셀 재실행)")
        print(f"   2. 몇 분 후 다시 시도")
        print(f"   3. AWS 콘솔에서 엔드포인트 로그 확인")
        TEST_SUCCESS = False
        
else:
    print("❌ 테스트 조건이 충족되지 않았습니다.")
    print(f"\n📋 현재 상태:")
    print(f"   - 엔드포인트 준비: {globals().get('ENDPOINT_READY', False)}")
    print(f"   - 테스트 데이터 준비: {globals().get('test_data_loaded', False)}")
    print(f"   - 엔드포인트 이름: {'있음' if 'endpoint_name' in globals() else '없음'}")
    
    if not globals().get('ENDPOINT_READY', False):
        print(f"\n💡 엔드포인트 상태 확인 셀을 다시 실행해주세요.")
    
    TEST_SUCCESS = False

# 테스트 결과 저장
globals()['TEST_SUCCESS'] = TEST_SUCCESS

❌ 테스트 조건이 충족되지 않았습니다.

📋 현재 상태:
   - 엔드포인트 준비: False
   - 테스트 데이터 준비: True
   - 엔드포인트 이름: 있음

💡 엔드포인트 상태 확인 셀을 다시 실행해주세요.


## 8. 엔드포인트 모니터링

배포된 엔드포인트의 상세 정보를 확인합니다.

In [9]:
# 엔드포인트 상세 정보 확인
if 'endpoint_name' in globals():
    print(f"📊 엔드포인트 모니터링: {endpoint_name}")
    print("=" * 50)
    
    try:
        # 엔드포인트 정보 조회
        endpoint_info = sm_client.describe_endpoint(EndpointName=endpoint_name)
        
        print(f"🔍 엔드포인트 상세 정보:")
        print(f"   - 이름: {endpoint_info['EndpointName']}")
        print(f"   - 상태: {endpoint_info['EndpointStatus']}")
        print(f"   - 생성 시간: {endpoint_info['CreationTime']}")
        
        if 'LastModifiedTime' in endpoint_info:
            print(f"   - 마지막 수정: {endpoint_info['LastModifiedTime']}")
        
        # Production Variants 정보
        if 'ProductionVariants' in endpoint_info:
            print(f"\n📋 Production Variants:")
            for variant in endpoint_info['ProductionVariants']:
                print(f"   - 이름: {variant['VariantName']}")
                print(f"   - 인스턴스 수: {variant['CurrentInstanceCount']}")
                print(f"   - 가중치: {variant['CurrentWeight']}")
        
        # 엔드포인트 구성 정보
        config_name = endpoint_info['EndpointConfigName']
        config_info = sm_client.describe_endpoint_config(EndpointConfigName=config_name)
        
        print(f"\n🔧 구성 정보:")
        print(f"   - 구성 이름: {config_name}")
        
        for variant in config_info['ProductionVariants']:
            print(f"   - 모델: {variant['ModelName']}")
            print(f"   - 인스턴스 타입: {variant['InstanceType']}")
            print(f"   - 초기 인스턴스 수: {variant['InitialInstanceCount']}")
        
        # 비용 정보
        print(f"\n💰 예상 비용 (ml.m5.large 기준):")
        print(f"   - 시간당: ~$0.10")
        print(f"   - 일일: ~$2.40")
        print(f"   - 월간: ~$72.00")
        
        # CloudWatch 메트릭 안내
        print(f"\n📈 모니터링 정보:")
        print(f"   - CloudWatch 메트릭: AWS/SageMaker/Endpoints")
        print(f"   - 주요 메트릭: Invocations, ModelLatency, OverheadLatency")
        print(f"   - 로그 그룹: /aws/sagemaker/Endpoints/{endpoint_name}")
        
    except Exception as e:
        print(f"❌ 엔드포인트 정보 조회 실패: {e}")
        
else:
    print("❌ 모니터링할 엔드포인트가 없습니다.")

📊 엔드포인트 모니터링: bank-marketing-endpoint-20260124-081832
🔍 엔드포인트 상세 정보:
   - 이름: bank-marketing-endpoint-20260124-081832
   - 상태: Creating
   - 생성 시간: 2026-01-24 08:18:36.574000+00:00
   - 마지막 수정: 2026-01-24 08:18:36.822000+00:00

🔧 구성 정보:
   - 구성 이름: bank-marketing-endpoint-20260124-081832
   - 모델: bank-marketing-aggressive-20260124-081832
   - 인스턴스 타입: ml.m5.large
   - 초기 인스턴스 수: 1

💰 예상 비용 (ml.m5.large 기준):
   - 시간당: ~$0.10
   - 일일: ~$2.40
   - 월간: ~$72.00

📈 모니터링 정보:
   - CloudWatch 메트릭: AWS/SageMaker/Endpoints
   - 주요 메트릭: Invocations, ModelLatency, OverheadLatency
   - 로그 그룹: /aws/sagemaker/Endpoints/bank-marketing-endpoint-20260124-081832


## 9. 리소스 정리

⚠️ **중요**: 비용 절약을 위해 사용하지 않는 엔드포인트는 반드시 삭제해야 합니다.

In [10]:
# 리소스 정리 안내
if 'endpoint_name' in globals():
    print("⚠️ 리소스 정리 안내")
    print("=" * 50)
    print(f"🔍 현재 실행 중인 리소스:")
    print(f"   - 엔드포인트: {endpoint_name}")
    print(f"   - 모델: {globals().get('model_name', 'N/A')}")
    print(f"   - 예상 일일 비용: ~$2.40")
    
    print(f"\n💰 비용 절약 옵션:")
    print(f"   1. 즉시 삭제 (권장): 아래 셀 실행")
    print(f"   2. 나중에 삭제: AWS 콘솔에서 수동 삭제")
    print(f"   3. 계속 사용: 프로덕션 또는 추가 테스트용")
    
    print(f"\n🔧 삭제 방법:")
    print(f"   - 노트북: 아래 삭제 셀 실행")
    print(f"   - AWS 콘솔: SageMaker > 추론 > 엔드포인트")
    print(f"   - AWS CLI: aws sagemaker delete-endpoint --endpoint-name {endpoint_name}")
    
    print(f"\n⏰ 권장사항:")
    print(f"   - 테스트 완료 후 즉시 삭제")
    print(f"   - 장기간 사용하지 않을 경우 삭제")
    print(f"   - 필요시 언제든 재배포 가능")
    
else:
    print("✅ 정리할 리소스가 없습니다.")

⚠️ 리소스 정리 안내
🔍 현재 실행 중인 리소스:
   - 엔드포인트: bank-marketing-endpoint-20260124-081832
   - 모델: bank-marketing-aggressive-20260124-081832
   - 예상 일일 비용: ~$2.40

💰 비용 절약 옵션:
   1. 즉시 삭제 (권장): 아래 셀 실행
   2. 나중에 삭제: AWS 콘솔에서 수동 삭제
   3. 계속 사용: 프로덕션 또는 추가 테스트용

🔧 삭제 방법:
   - 노트북: 아래 삭제 셀 실행
   - AWS 콘솔: SageMaker > 추론 > 엔드포인트
   - AWS CLI: aws sagemaker delete-endpoint --endpoint-name bank-marketing-endpoint-20260124-081832

⏰ 권장사항:
   - 테스트 완료 후 즉시 삭제
   - 장기간 사용하지 않을 경우 삭제
   - 필요시 언제든 재배포 가능


In [11]:
# 엔드포인트 삭제 (주석 해제하여 실행)
# ⚠️ 주의: 이 셀을 실행하면 엔드포인트가 영구적으로 삭제됩니다!

DELETE_ENDPOINT = False  # True로 변경하면 삭제 실행

if DELETE_ENDPOINT and 'endpoint_name' in globals():
    print(f"🗑️ 엔드포인트 삭제 중: {endpoint_name}")
    
    try:
        # 엔드포인트 삭제
        sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"✅ 엔드포인트 삭제 요청 완료")
        
        # 엔드포인트 구성도 삭제 (선택사항)
        try:
            endpoint_info = sm_client.describe_endpoint(EndpointName=endpoint_name)
            config_name = endpoint_info['EndpointConfigName']
            
            # 잠시 대기 후 구성 삭제
            time.sleep(10)
            sm_client.delete_endpoint_config(EndpointConfigName=config_name)
            print(f"✅ 엔드포인트 구성 삭제 완료: {config_name}")
            
        except Exception as e:
            print(f"⚠️ 엔드포인트 구성 삭제 실패: {e}")
        
        print(f"\n💰 비용 절약 완료!")
        print(f"   - 더 이상 비용이 발생하지 않습니다")
        print(f"   - 필요시 언제든 재배포 가능합니다")
        
    except Exception as e:
        print(f"❌ 엔드포인트 삭제 실패: {e}")
        
elif not DELETE_ENDPOINT:
    print("💡 엔드포인트를 삭제하려면:")
    print("   1. 위의 셀에서 DELETE_ENDPOINT = True로 변경")
    print("   2. 셀을 다시 실행")
    
else:
    print("❌ 삭제할 엔드포인트가 없습니다.")

💡 엔드포인트를 삭제하려면:
   1. 위의 셀에서 DELETE_ENDPOINT = True로 변경
   2. 셀을 다시 실행


## 10. 배포 결과 요약

In [12]:
# 최종 결과 요약
print("📋 모델 배포 및 테스트 결과 요약")
print("=" * 60)

# 배포 상태
deployment_started = globals().get('DEPLOYMENT_STARTED', False)
endpoint_ready = globals().get('ENDPOINT_READY', False)
test_success = globals().get('TEST_SUCCESS', False)

print(f"🚀 배포 상태:")
print(f"   - 배포 시작: {'✅ 성공' if deployment_started else '❌ 실패'}")
print(f"   - 엔드포인트 준비: {'✅ 완료' if endpoint_ready else '❌ 미완료'}")
print(f"   - 테스트 실행: {'✅ 성공' if test_success else '❌ 실패'}")

if 'endpoint_name' in globals():
    print(f"\n📊 배포된 리소스:")
    print(f"   - 엔드포인트: {endpoint_name}")
    print(f"   - 모델: {globals().get('best_model_name', 'N/A')}")
    print(f"   - 타입: {globals().get('model_type', 'N/A')}")
    print(f"   - 인스턴스: ml.m5.large × 1개")

if test_success:
    print(f"\n🧪 테스트 결과:")
    print(f"   - 예측 성공: ✅")
    print(f"   - 엔드포인트 정상 작동 확인")

print(f"\n💰 비용 정보:")
if endpoint_ready:
    print(f"   - 현재 상태: 🔴 과금 중")
    print(f"   - 예상 비용: ~$2.40/일")
    print(f"   - ⚠️ 사용 완료 후 반드시 삭제하세요!")
else:
    print(f"   - 현재 상태: 🟢 과금 없음")

print(f"\n🔗 다음 단계:")
if endpoint_ready:
    print(f"   - 실제 데이터로 추가 테스트")
    print(f"   - CloudWatch로 성능 모니터링")
    print(f"   - 애플리케이션 통합")
    print(f"   - 사용 완료 후 리소스 정리")
else:
    print(f"   - 배포 문제 해결")
    print(f"   - 엔드포인트 상태 재확인")
    print(f"   - 필요시 재배포")

print("\n" + "=" * 60)
if endpoint_ready and test_success:
    print("🎉 모델 배포 및 테스트 완료!")
elif deployment_started:
    print("⏳ 배포 진행 중... 상태 확인 셀을 다시 실행해주세요.")
else:
    print("❌ 배포 실패. 위의 단계들을 다시 확인해주세요.")

📋 모델 배포 및 테스트 결과 요약
🚀 배포 상태:
   - 배포 시작: ✅ 성공
   - 엔드포인트 준비: ❌ 미완료
   - 테스트 실행: ❌ 실패

📊 배포된 리소스:
   - 엔드포인트: bank-marketing-endpoint-20260124-081832
   - 모델: 적극적 하이퍼파라미터 모델
   - 타입: aggressive
   - 인스턴스: ml.m5.large × 1개

💰 비용 정보:
   - 현재 상태: 🟢 과금 없음

🔗 다음 단계:
   - 배포 문제 해결
   - 엔드포인트 상태 재확인
   - 필요시 재배포

⏳ 배포 진행 중... 상태 확인 셀을 다시 실행해주세요.
